### Memories

LLM기반 애플리케이션에서 기억은 애플리케이션이 장단기적으로 사용자 상호 작용에 대한 참조를 유지할 수 있게 해 준다. Chat GPT를 사용할 때를 떠올려 보자. 사용자는 Chat GPT와 상호 작용하는 동안 모델에 명시적으로 말하지 않고도 이전 상호 작용을 참조해 후속 질문을 할 수 있다. 또한 모든 대화는 타래에 저장되므로, 이전 대화를 이어서 하고 싶을 경우 모든 문맥을 Chat GPT에 제공할 필요 없이 타래를 다시 열기만 하면 된다. 이는 사용자의 상호 작용을 기억 변수에 저장하고 후속 질문을 처리하는 동안 이 기억을 문맥으로 사용할 수 있는 Chat GPT의 기능 덕분에 가능하다.

langchain은 애플리케이션 내에서 기억 시스템을 설계할 수 있는 여러 모듈을 제공해 읽기, 쓰기 기능을 모두 지원한다. 기억 시스템에서 해야 할 첫 번째 단계는 인간과의 상호 작용을 실제로 어딘가에 저장하는 것이다. 이를 위해 다양한 기억 시스템을 Redis, Cassandra, Postgres등 다양한 서드파티와 통합해 활용할 수 있다. 그 다음, 기억 시스템에 질의하는 방법을 정의할 때 활용할 수 있는 다양한 기억 유형이 있다.

- conversation buffer memory
- conversation buffer window memory
- entity memory
- conversation knowledge graph memory
- conversation summary memory
- conversation summary buffer memory
- conversation token buffer memory
- vector store-backed memory

In [1]:
import numpy as np
import pandas as pd
import openai
from openai import OpenAI
import os

with open('../config/api.key') as file :
    lines = file.readlines()
    api_key = lines[0].strip()
    serp_api_key = lines[1].strip()
    langsmith_api_key = lines[2].strip()

openai.api_key = api_key
os.environ['OPENAI_API_KEY'] = openai.api_key

In [5]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_openai import OpenAI

memory = ConversationSummaryMemory(llm = OpenAI(temperature = 0))
memory.save_context(
    {'input': '2026년에 미국 주식 시장 소형주 투자를 하고 싶은데 투자할만한 종목이 있을까? 퀀텀컴퓨팅 및 AI와 관련된 종목 위주로 했으면 좋겠어.'},
    {'output': '2026년 투자할 만한 미국 소형주에는 Spectral AI(ticker : MDAI)가 있습니다.'}
)
memory.load_memory_variables({})

{'history': '\nThe human asks for investment advice for the US small-cap market in 2025, specifically looking for stocks related to quantum computing and AI. The AI suggests Spectral AI (ticker: MDAI) as a potential option.'}

메모리는 초기화한 OpenAI LLM을 활용해 대화를 요약하였다. 애플리케이션 내에서 어떤 기억을 사용할지 정해진 바는 없지만, 특정 기억이 특히 적합한 몇 가지 시나리오가 있다. 예를 들어, 지식 그래프 기억은 대규모의 다양한 데이터 말뭉치에서 정보에 액세스하고 의미론적 관게에 기반해 응답을 생성해야 하는 애플리케이션에 유용하며, 대화 요약 버퍼 기억은 여러 차례에 걸쳐 일관된 문맥을 유지하면서 이전 대화 이력을 압축 및 요약할 수 있는 대화형 에이전트를 만드는 데 적합할 수 있다.